In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from datetime import datetime
import seaborn as sns
%matplotlib inline

In [2]:
weather=pd.read_csv('austin_weather.csv').drop(['DewPointHighF','DewPointLowF','HumidityHighPercent','HumidityLowPercent',
                                               'SeaLevelPressureHighInches','SeaLevelPressureLowInches','VisibilityHighMiles',
                                               'VisibilityLowMiles','WindHighMPH','WindGustMPH','HumidityAvgPercent'],1)
trips=pd.read_csv('austin_bikeshare_trips.csv')
station=pd.read_csv('austin_bikeshare_stations_w_zipcode.csv')

In [3]:
weather['Rain']=np.where(weather.Events.str.contains('Rain'), 1, 0)
weather['Fog']=np.where(weather.Events.str.contains('Fog'), 1, 0)
weather['Thunderstorm']=np.where(weather.Events.str.contains('Thunderstorm'), 1, 0)
weather['Snow']=np.where(weather.Events.str.contains('Snow'), 1, 0)
weather['None']=np.where(weather.Events.isnull, 1, 0)
weather.Date=pd.to_datetime(weather.Date)
weather=weather.set_index('Date').drop('Events',1)

In [5]:
ind = pd.DatetimeIndex(trips.start_time)
trips['Date'] = ind.date.astype('datetime64')
trips['hour'] = ind.hour
by_date = trips.pivot_table('trip_id', aggfunc='count',index='Date',columns='subscriber_type', )
by_date = trips.groupby(['Date', 'subscriber_type'])['trip_id'].count()
by_date.name = 'count'
by_date = by_date.reset_index('subscriber_type')
trips_2=trips[trips.year.isin([2014,2015,2016])]
trip_weather=trips_2.merge(weather,left_on='Date',right_on='Date',
                         suffixes=['_trips','_weather']).drop(['SeaLevelPressureAvgInches'],1)

In [6]:
trip_weather["start_time"] = trip_weather["start_time"].map(lambda x : datetime.strptime(x, "%Y/%m/%d %H:%M"))
trip_weather["hours"] = trip_weather["start_time"].map(lambda x : x.hour)
val=''
def time_of_day(x):
    global val
    if (x>=6)&(x<10):
        val='morning rush hours'
    elif (x>=15)&(x<19):
        val='afternoon ruch hours'
    elif (x>=10)&(x<15):
        val='noon'
    elif (x>=19)|(x<6):
        val='night'
    return val
trip_weather['Time of the day'] = trip_weather['hours'].apply(lambda a: time_of_day(a))

In [7]:
trip_weather.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,...,VisibilityAvgMiles,WindAvgMPH,PrecipitationSumInches,Rain,Fog,Thunderstorm,Snow,None,hours,Time of the day
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,...,8,5,0,0,0,0,0,1,19,night
1,62.0,16:12:00,22,2564.0,5th & San Marcos,3.0,2499.0,City Hall / Lavaca & 2nd,2015-03-19 16:12:00,Walk Up,...,8,5,0,0,0,0,0,1,16,afternoon ruch hours
2,195.0,20:12:00,29,1008.0,Nueces @ 3rd,3.0,2563.0,Davis at Rainey Street,2015-03-19 20:12:00,Walk Up,...,8,5,0,0,0,0,0,1,20,night
3,728.0,15:12:00,31,2539.0,Convention Center / 3rd & Trinity,3.0,2711.0,Barton Springs @ Kinney Ave,2015-03-19 15:12:00,Walk Up,...,8,5,0,0,0,0,0,1,15,afternoon ruch hours
4,920.0,17:12:00,247,2822.0,East 6th at Robert Martinez,3.0,2575.0,Riverside @ S. Lamar,2015-03-19 17:12:00,Walk Up,...,8,5,0,0,0,0,0,1,17,afternoon ruch hours
